## qLDPC CSS-T Codes
Generating the corresponding generator matrices for the the binary codes of the $Q(C_1, C_2)$ code. Based on Theorem 9 of the paper *Toward Quantum CSS-T Codes from Sparce Matrices*

In [41]:
import sympy as sp
import numpy as np

def schur(A: sp.Matrix, B: sp.Matrix) -> sp.Matrix:
    if A.shape != B.shape:
        raise ValueError("Matrices must have same shape.")
    return A.multiply_elementwise(B)

def square(A: sp.Matrix) -> sp.Matrix:
    m, n = A.shape
    C = sp.zeros((m * m, n))

    l = 0
    for i in range(m):
        for j in range(m):
            C[l, :] = schur(A.row(i), A.row(j))  
            l += 1
    return C

def QCLD(L: int, A: sp.Matrix) -> sp.Matrix:
    P = sp.zeros(L)
    for i in range(L - 1):
        P[i, i + 1] = 1
    P[L - 1, 0] = 1

    m, n = A.shape
    H = sp.zeros(m * L, n * L)
    for i in range(m):
        for j in range(n):
            exp = int(A[i, j]) % L
            if exp == -1:
                block = sp.zeros(L)
            else:
                block = (P ** exp) % 2
            H[i*L:(i+1)*L, j*L:(j+1)*L] = block

    return H % 2

In [ ]:
def gf2_rref(M):
    M = M.applyfunc(lambda x: int(x) & 1)
    rows, cols = M.shape
    r = 0
    for c in range(cols):
        pivot = next((i for i in range(r, rows) if M[i, c] == 1), None)
        if pivot is None:
            continue
        if pivot != r:
            M.row_swap(pivot, r)
        for i in range(rows):
            if i != r and M[i, c] == 1:
                M.row_op(i, lambda v, j: (v + M[r, j]) % 2)
        r += 1
        if r == rows:
            break
    nonzero_rows = [M.row(i) for i in range(M.rows) if any(M.row(i))]
    return sp.Matrix(nonzero_rows)

In [45]:
L = 4
A = sp.Matrix([
    [3, 1, 2, 1],
    [3, 3, 2, 3]
])

G = QCLD(L, A)
G_rref = gf2_rref(G)
sp.pprint(G_rref)

⎡1  0  0  0  0  0  1  0  0  0  0  1  0  0  1  0⎤
⎢                                              ⎥
⎢0  1  0  0  0  0  0  1  1  0  0  0  0  0  0  1⎥
⎢                                              ⎥
⎢0  0  1  0  0  0  1  0  0  1  0  0  0  0  1  0⎥
⎢                                              ⎥
⎢0  0  0  1  0  0  0  1  0  0  1  0  0  0  0  1⎥
⎢                                              ⎥
⎢0  0  0  0  1  0  1  0  0  0  0  0  1  0  1  0⎥
⎢                                              ⎥
⎣0  0  0  0  0  1  0  1  0  0  0  0  0  1  0  1⎦
